In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-06 03:11:17,374 INFO: Initializing external client
2025-03-06 03:11:17,374 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:11:18,186 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680
Fetching data from 2025-02-05 08:11:17.373429+00:00 to 2025-03-06 07:11:17.373429+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.27s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-05 09:00:00+00:00,2,0
1,2025-02-05 10:00:00+00:00,2,0
2,2025-02-05 11:00:00+00:00,2,0
3,2025-02-05 12:00:00+00:00,2,0
4,2025-02-05 13:00:00+00:00,2,0
...,...,...,...
175135,2025-03-06 03:00:00+00:00,263,4
175136,2025-03-06 04:00:00+00:00,263,6
175137,2025-03-06 05:00:00+00:00,263,39
175138,2025-03-06 06:00:00+00:00,263,73


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175140 entries, 0 to 176147
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         175140 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  175140 non-null  int32                  
 2   rides               175140 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175140 entries, 0 to 176147
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175140 non-null  datetime64[us]
 1   pickup_location_id  175140 non-null  int32         
 2   rides               175140 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,48,145,8,155,145,62,60,6,80,...,155,127,154,107,72,31,34,108,263,2025-02-15 17:00:00
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,2,0,210,2025-02-10 03:00:00
2,3,0,0,0,0,0,1,0,2,0,...,1,0,0,0,2,0,0,0,216,2025-02-07 14:00:00
3,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,217,2025-03-05 13:00:00
4,0,8,1,0,6,10,9,3,7,6,...,47,2,2,20,2,37,41,4,261,2025-03-05 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2,2,1,1,3,0,0,1,1,0,...,3,2,1,2,4,2,13,5,97,2025-02-05 22:00:00
248,0,2,1,1,0,0,0,5,0,1,...,0,0,1,0,0,0,1,1,82,2025-02-28 15:00:00
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,73,2025-02-15 18:00:00
250,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,160,2025-02-13 13:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-06 03:11:27,019 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 03:11:27,021 INFO: Initializing external client
2025-03-06 03:11:27,021 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:11:27,670 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680
Fetching data from 2025-02-05 08:11:27.019449+00:00 to 2025-03-06 07:11:27.019449+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.28s) 


In [12]:
current_date

Timestamp('2025-03-06 08:11:27.019449+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-05 09:00:00
1,0,0,0,1,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,3,2025-03-05 09:00:00
2,2,3,2,0,1,3,2,1,1,3,...,3,3,0,0,0,1,8,15,4,2025-03-05 09:00:00
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-05 09:00:00
4,2,1,4,1,2,1,2,2,2,1,...,0,2,0,2,0,2,1,7,7,2025-03-05 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,259,2025-03-05 09:00:00
248,1,0,3,2,5,0,1,2,0,0,...,0,0,0,1,2,0,0,3,260,2025-03-05 09:00:00
249,14,12,23,23,25,28,23,29,40,60,...,1,0,0,0,2,0,4,15,261,2025-03-05 09:00:00
250,158,114,84,84,65,73,99,89,115,107,...,2,2,4,7,18,79,171,196,262,2025-03-05 09:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 03:11:36,239 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 03:11:36,240 INFO: Initializing external client
2025-03-06 03:11:36,241 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:11:36,815 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,10.0
3,6,0.0
4,7,3.0
...,...,...
247,259,0.0
248,260,1.0
249,261,17.0
250,262,177.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 237, 186, 162, 239, 170, 238, 161, 142, 140], dtype=int32)